<a href="https://colab.research.google.com/github/ibahas/Uploadee-Google-Drive/blob/main/Colab_Multi_Destination_File_Uploader_(Upload_ee_%26_Google_Drive).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#          Colab Multi-Destination File Uploader (Upload.ee & Google Drive)
# ======================================================================


1.   List item
2.   List item



This notebook provides an interactive file browser and uploader GUI within Google Colaboratory.

**Features:**
*   Browse `/content/`.
*   Select multiple files.
*   Upload selected files sequentially to **Upload.ee** or **Google Drive**.
*   Progress bar for current file upload.
*   Status logging.

**Instructions:**
1.  Run the **"Install Dependencies"** cell below first.
2.  Run the **"Main Application Code"** cell to launch the GUI.
3.  Follow the usage instructions within the application's output area or the README.md if available.

# Install necessary libraries (run this cell first)
!pip install --upgrade requests google-api-python-client google-auth-httplib2 google-auth-oauthlib --quiet
print("Dependencies installed/updated.")

---


## Main Application Code

Run the following cell to start the file browser and uploader interface. The UI widgets will appear below this cell after execution.

In [ ]:
# ======================================================================
# Colab File Browser & Multi-Destination Uploader (Multiple Files)
# v4.12 - Display Upload.ee link on success
# ======================================================================

# --- Imports ---
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import html
import subprocess
import shlex
import re
import requests
import time
import json
from functools import partial # For cleaner event handlers

# --- Google Drive Specific Imports ---
try:
    from google.colab import auth as colab_auth
    from google.auth import default as google_auth_default
    from googleapiclient.discovery import build as build_google_service
    from googleapiclient.errors import HttpError
    from googleapiclient.http import MediaFileUpload
    google_libs_available = True
except ImportError:
    google_libs_available = False
    print("⚠️ Warning: Google Drive libraries not found. Run 'pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib'")

# --- Global State ---
current_path = '/content/'
selected_files_list = [] # Use a list for multiple selections
# --- Google Drive State ---
drive_service = None
drive_authenticated = False
# --- UI References ---
file_checkboxes = {} # Dictionary to store checkboxes {path: checkbox_widget}

# ======================================================
#  Helper Function: Get Upload.ee Details
# ======================================================
# (Unchanged from previous working version)
def get_upload_ee_details():
    """
    Connects to upload.ee to fetch dynamic upload ID, URL, and cookies.
    Returns: tuple: (dynamic_upload_url, cookie_string, upload_id) or (None, None, None) on failure.
    """
    # print("Attempting to fetch Upload.ee dynamic upload ID...") # Less verbose
    session = requests.Session()
    link_script_url = "https://www.upload.ee/ubr_link_upload.php"
    base_url = "https://www.upload.ee/"
    rnd_id = int(time.time() * 1000); params = {'rnd_id': rnd_id}
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36', 'Accept-Language': 'en-US,en;q=0.9', }
    session.headers.update(headers)
    try: # Pre-load main page for cookies
        preload_headers = { 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Sec-Fetch-Dest': 'document','Sec-Fetch-Mode': 'navigate', 'Sec-Fetch-Site': 'same-origin', 'Upgrade-Insecure-Requests': '1', 'Referer': base_url, }
        session.headers.update(preload_headers); main_page_response = session.get(base_url, timeout=15); main_page_response.raise_for_status()
    except Exception as e: print(f"⚠️ Warn: Failed pre-fetch: {e}") # Continue even if pre-fetch fails
    script_headers = { 'Accept': '*/*', 'Referer': base_url, 'Sec-Fetch-Dest': 'script', 'Sec-Fetch-Mode': 'no-cors', 'Sec-Fetch-Site': 'same-origin', }
    session.headers.update(script_headers); upload_id = None
    try: # Fetch Upload ID
        response = session.get(link_script_url, params=params, timeout=15); response.raise_for_status()
        match = re.search(r'startUpload\("([a-f0-9]+)"\s*,\s*0\s*\);', response.text)
        if match: upload_id = match.group(1);
        else: print("❌ Failed to find upload_id pattern."); return None, None, None
        dynamic_upload_url = f"https://www.upload.ee/cgi-bin/ubr_upload.pl?X-Progress-ID={upload_id}&upload_id={upload_id}"
        cookie_dict = session.cookies.get_dict(); cookie_string = "; ".join([f"{k}={v}" for k, v in cookie_dict.items()])
        return dynamic_upload_url, cookie_string, upload_id
    except Exception as e: print(f"❌ Error fetching Upload.ee ID: {type(e).__name__} - {e}"); return None, None, None

# ======================================================
#  Helper Function: Authenticate Google Drive
# ======================================================
# (Unchanged from previous working version)
def authenticate_google_drive():
    """Handles Google Drive authentication using Colab."""
    global drive_service, drive_authenticated, output_area
    if not google_libs_available:
        with output_area: print("❌ Google libraries not installed."); return False
    try:
        with output_area: clear_output(wait=True); print("Authenticating Google Drive via Colab...")
        colab_auth.authenticate_user()
        creds, _ = google_auth_default()
        drive_service = build_google_service('drive', 'v3', credentials=creds)
        drive_authenticated = True
        with output_area: clear_output(wait=True); print("✅ Google Drive Authentication Successful!")
        return True
    except Exception as e:
        drive_authenticated = False; drive_service = None
        with output_area: clear_output(wait=True); print(f"❌ Google Drive Authentication Failed: {type(e).__name__} - {e}")
        return False

# ======================================================
#  Helper Function: Upload to Google Drive
# ======================================================
# (Unchanged from previous working version)
def perform_google_drive_upload(file_path, file_name, target_folder_id):
    """Uploads the specified file to Google Drive."""
    global drive_service, drive_authenticated, output_area, progress_bar
    if not drive_authenticated or drive_service is None: print("⚠️ Please Authenticate Google Drive first!"); return False
    print(f"\nStarting Google Drive upload for: {file_name}")
    progress_bar.value = 0; progress_bar.description = f'GDrive: {file_name[:20]}...'; progress_bar.bar_style = 'info'; progress_bar.layout.display = 'block'
    try:
        file_metadata = {'name': file_name}
        if target_folder_id:
             if re.match(r'^[a-zA-Z0-9_-]{10,}$', target_folder_id): file_metadata['parents'] = [target_folder_id]; print(f"   Targeting Folder ID: {target_folder_id}")
             else: print(f"⚠️ Invalid Folder ID format: '{target_folder_id}'. Uploading to root.")
        else: print("   No Folder ID specified. Uploading to root 'My Drive'.")
        media = MediaFileUpload(file_path, chunksize=1024*1024*5, resumable=True)
        request = drive_service.files().create(body=file_metadata, media_body=media, fields='id, name, webViewLink')
        response = None; print("   Uploading..."); last_progress_percent = 0
        while response is None:
            status, response = request.next_chunk()
            if status: percent = int(status.progress() * 100); progress_bar.value = percent
        progress_bar.value = 100 # Ensure 100%
        file_id = response.get('id'); file_name_uploaded = response.get('name'); file_link = response.get('webViewLink')
        print(f"✅ GDrive Upload Successful: {file_name_uploaded} ({file_id})"); print(f"   View Link: {file_link}")
        progress_bar.bar_style = 'success'; return True
    except HttpError as error:
        print(f"❌ Google Drive API Error for {file_name}: {error}")
        try: error_details = json.loads(error.content).get('error'); print(f"   Reason: {error_details.get('message')} (Code: {error_details.get('code')})")
        except Exception: pass
        progress_bar.bar_style = 'danger'; return False
    except Exception as e: print(f"❌ Unexpected Error during GDrive upload for {file_name}: {type(e).__name__} - {e}"); progress_bar.bar_style = 'danger'; return False

# ======================================================
#  Helper Function: Upload to Upload.ee (Returns Link)
# ======================================================
def perform_upload_ee_upload(file_path, file_name, upload_url, cookie_header_value, upload_id):
    """
    Handles the Upload.ee upload process with progress polling.
    Returns the 'finished page' URL string on success, None on failure.
    """
    global output_area, progress_bar

    print(f"\nStarting Upload.ee upload for: {file_name}")
    progress_bar.value = 0; progress_bar.description = f'Upload.ee: {file_name[:18]}...'; progress_bar.bar_style = 'info'; progress_bar.layout.display = 'block'
    upload_process = None; success_status = False; final_url = None

    try:
        command = [ 'curl', '-L', upload_url, '-H', 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', '-H', 'Accept-Language: en-US,en;q=0.9', '-H', 'Cache-Control: max-age=0', '-H', 'Origin: https://www.upload.ee', '-H', 'Referer: https://www.upload.ee/', '-H', 'Sec-Fetch-Dest: iframe', '-H', 'Sec-Fetch-Mode: navigate', '-H', 'Sec-Fetch-Site: same-origin', '-H', 'Sec-Fetch-User: ?1', '-H', 'Upgrade-Insecure-Requests: 1', '-H', 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36', '-F', f'upfile_0=@{file_path};filename={file_name}', '-F', 'link=', '-F', 'email=', '-F', 'category=cat_file', '-F', 'big_resize=none', '-F', 'small_resize=120x90' ]
        if cookie_header_value: command.extend(['-b', cookie_header_value])
        user_agent_string = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'
        print("--- Starting Background Upload ---"); upload_process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding='utf-8', errors='ignore')
        progress_url = "https://www.upload.ee/progress"
        progress_headers = {'User-Agent': user_agent_string,'Accept': '*/*', 'Referer': 'https://www.upload.ee/?', 'X-Progress-ID': upload_id,'Sec-Fetch-Dest': 'empty','Sec-Fetch-Mode': 'cors','Sec-Fetch-Site': 'same-origin',}
        progress_session = requests.Session(); progress_session.headers.update(progress_headers)
        if cookie_header_value:
            try: cookies_for_requests = {c.split('=')[0].strip(): c.split('=',1)[1].strip() for c in cookie_header_value.split(';') if '=' in c}; progress_session.cookies.update(cookies_for_requests)
            except Exception: pass
        last_progress = 0
        while upload_process.poll() is None:
            try:
                prog_response = progress_session.get(progress_url, timeout=5)
                if prog_response.status_code == 200:
                    try: data = prog_response.json(); state = data.get("state"); received = data.get("received"); size = data.get("size")
                    except (json.JSONDecodeError, AttributeError, TypeError): continue
                    if state == "uploading" and isinstance(received, int) and isinstance(size, int) and size > 0: percent = min(100.0, (received / size) * 100.0); progress_bar.value = percent
                    elif state == "done": break
                    elif state == "error": break
            except requests.exceptions.RequestException: pass
            time.sleep(1.5)
        print("--- Background Upload Finished ---"); progress_bar.value = 100
        stdout, stderr = upload_process.communicate(); return_code = upload_process.returncode
        server_error_found = False
        if stdout:
             success_pattern = r"parent\.location\.href='(https://www\.upload\.ee/\?page=finished&upload_id=([a-f0-9]+))'"
             match = re.search(success_pattern, stdout)
             if match:
                 success_status = True
                 final_url = match.group(1) # <<< Store the captured URL
             elif "Internal Server Error" in stdout or "Software error" in stdout: server_error_found = True
             elif "Forbidden" in stdout or "Error 403" in stdout: server_error_found = True
             elif "Payload Too Large" in stdout or "Error 413" in stdout: server_error_found = True

        # --- Updated Verdict Logic ---
        if success_status:
            print(f"✅ Upload.ee Successful: {file_name}")
            progress_bar.bar_style = 'success'
            return final_url # <<< RETURN URL ON SUCCESS
        elif server_error_found:
            print(f"❌ Upload.ee Failed (Server Error): {file_name}")
            progress_bar.bar_style = 'danger'
        elif return_code != 0:
            print(f"❌ Upload.ee Failed (Curl Error {return_code}): {file_name}")
            progress_bar.bar_style = 'danger'
        else:
            print(f"⚠️ Upload.ee Status Uncertain: {file_name}")
            progress_bar.bar_style = 'warning'

        return None # <<< RETURN None ON FAILURE/UNCERTAIN

    except Exception as e:
        print(f"❌ Unexpected Error during Upload.ee for {file_name}: {type(e).__name__} - {e}")
        if upload_process and upload_process.poll() is None:
             try: upload_process.kill(); upload_process.communicate()
             except Exception as kill_e: print(f"   (Error terminating process: {kill_e})")
        progress_bar.bar_style = 'danger'
        return None # <<< RETURN None ON EXCEPTION

# ======================================================
#  File Browser UI & Logic
# ======================================================
# (Unchanged)
def update_directory_view(path_to_display):
    global current_path, selected_files_list, output_area, dir_list_box, current_path_label
    global file_checkboxes, select_all_button, deselect_all_button # Need controls
    try: path_to_display = os.path.abspath(path_to_display)
    except Exception as e:
        with output_area: clear_output(wait=True); print(f"❌ Error resolving path: {e}")
        path_to_display = current_path if os.path.exists(current_path) else '/content/'
    if not path_to_display.startswith('/content'):
        with output_area: clear_output(wait=True); print(f"⚠️ Navigation restricted.")
        path_to_display = '/content/'
    if not os.path.exists(path_to_display) or not os.path.isdir(path_to_display):
         parent = os.path.dirname(current_path); path_to_display = parent if os.path.exists(parent) and os.path.isdir(parent) else '/content/'
         with output_area: clear_output(wait=True); print(f"ℹ️ Path not found/not dir. Resetting view.")
    current_path = path_to_display; current_path_label.value = f"<b>Current:</b> {html.escape(current_path)}"
    items = []; file_checkboxes.clear()
    try: items = sorted(os.listdir(current_path), key=str.lower)
    except PermissionError:
        with output_area: clear_output(wait=True); print(f"❌ Permission denied: {current_path}")
        dir_list_box.children = []; select_all_button.disabled=True; deselect_all_button.disabled=True; return
    except Exception as e:
        with output_area: clear_output(wait=True); print(f"❌ Error listing dir: {e}")
        dir_list_box.children = []; select_all_button.disabled=True; deselect_all_button.disabled=True; return
    new_children = []; current_dir_has_files = False
    item_layout = widgets.Layout(min_width='150px', max_width='200px', margin='5px', padding='5px', border='1px solid #ccc', display='flex', flex_direction='column', align_items='center')
    folder_button_layout = widgets.Layout(min_width='150px', max_width='200px', margin='5px', padding='5px', border='1px solid #a0a0ff', background_color='#e8e8ff')
    checkbox_layout = widgets.Layout(width='auto', margin='0 0 3px 0')
    label_layout = widgets.Layout(width='100%', text_align='center')
    if current_path != '/' and current_path != '/content': # Add "Go Up" Button
        parent_path = os.path.dirname(current_path)
        if parent_path.startswith('/content') or parent_path == '/':
             up_button = widgets.Button(description="⬆️ ..", layout=folder_button_layout, button_style='info'); up_button.tooltip = f"Go to: {parent_path}"
             up_button.on_click(lambda b, path=parent_path: update_directory_view(path)); new_children.append(up_button)
    for item in items: # Add Folders (as Buttons)
        item_path = os.path.join(current_path, item)
        try:
            if os.path.isdir(item_path):
                 folder_button = widgets.Button(description=f"📁 {item}", layout=folder_button_layout, button_style='primary'); folder_button.tooltip = f"Open: {item_path}"
                 folder_button.on_click(lambda b, path=item_path: update_directory_view(path)); new_children.append(folder_button)
        except Exception as e: print(f"⚠️ Warn: Cannot process folder {item}: {e}")
    for item in items: # Add Files (as Checkboxes + Labels)
        item_path = os.path.join(current_path, item)
        try:
            if not os.path.isdir(item_path) and os.path.exists(item_path):
                current_dir_has_files = True; display_name = item if len(item) < 20 else item[:17] + '...'
                file_label = widgets.Label(f"{display_name}", layout=label_layout, tooltip=item)
                checkbox = widgets.Checkbox(value=(item_path in selected_files_list), description='', indent=False, layout=checkbox_layout)
                file_checkboxes[item_path] = checkbox
                checkbox.observe(partial(handle_file_checkbox_change, item_path), names='value')
                file_entry = widgets.VBox([ widgets.HTML("📄", layout=widgets.Layout(margin='2px')), checkbox, file_label ], layout=item_layout)
                new_children.append(file_entry)
        except Exception as e: print(f"⚠️ Warn: Cannot process file {item}: {e}")
    dir_list_box.children = tuple(new_children)
    dir_list_box.layout.display = 'flex'; dir_list_box.layout.flex_flow = 'row wrap'; dir_list_box.layout.align_content = 'flex-start'; dir_list_box.layout.align_items = 'stretch'
    select_all_button.disabled = not current_dir_has_files; deselect_all_button.disabled = not current_dir_has_files

# --- UI Helper functions ---
# (Unchanged)
def update_selected_files_viewer():
    global selected_files_list, selected_files_viewer
    sorted_list = sorted([os.path.basename(p) for p in selected_files_list])
    selected_files_viewer.options = sorted_list; selected_files_viewer.value = tuple(sorted_list)
def handle_file_checkbox_change(path, change):
    global selected_files_list
    if change['new'] == True:
        if path not in selected_files_list: selected_files_list.append(path)
    else:
        if path in selected_files_list: selected_files_list.remove(path)
    update_selected_files_viewer()
def handle_select_all(b):
    global file_checkboxes
    for cb in file_checkboxes.values():
        if not cb.value: cb.value = True
def handle_deselect_all(b):
    global file_checkboxes
    for cb in file_checkboxes.values():
        if cb.value: cb.value = False

# ======================================================
#  Main Upload Button Handler (Router) - Updated for Link Display
# ======================================================
def handle_confirm_and_upload_click(b):
    global selected_files_list, output_area, progress_bar
    global upload_destination_dropdown, google_auth_button, gdrive_folder_id_input

    with output_area:
        try:
            clear_output(wait=True)
            if not selected_files_list: print("⚠️ No files selected for upload."); return

            destination = upload_destination_dropdown.value
            files_to_upload = list(selected_files_list) # Create copy
            total_files = len(files_to_upload)
            print(f"▶️ Preparing to upload {total_files} file(s) to {destination}...\n" + "-" * 20)

            target_folder_id_str = None
            if destination == 'Google Drive':
                if not google_libs_available: print("❌ Google libraries unavailable."); return
                if not drive_authenticated: print("✋ Please authenticate Google Drive first."); return
                target_folder_id_str = gdrive_folder_id_input.value.strip()

            confirm_and_upload_button.disabled = True; google_auth_button.disabled = True
            upload_success_count = 0; upload_fail_count = 0

            # Main Upload Loop
            for i, file_path in enumerate(files_to_upload):
                file_name = os.path.basename(file_path)
                print(f"\n--- Uploading file {i+1} of {total_files}: {file_name} ---")
                progress_bar.value = 0; progress_bar.layout.display = 'block'; upload_successful = False
                result_info = None # To store link or boolean

                try:
                    if destination == 'Upload.ee':
                        dynamic_upload_url, dynamic_cookie_string, upload_id = get_upload_ee_details()
                        if dynamic_upload_url and upload_id:
                            # <<< Store result (URL or None) >>>
                            result_info = perform_upload_ee_upload(file_path, file_name, dynamic_upload_url, dynamic_cookie_string, upload_id)
                            # <<< Check if result is a URL string for success >>>
                            upload_successful = isinstance(result_info, str)
                            if upload_successful:
                                print(f"   Finished Page URL: {result_info}") # Print the link
                        else:
                            print(f"❌ ERROR: Failed to get Upload.ee details for {file_name}.")
                            upload_successful = False # Explicitly set failure

                    elif destination == 'Google Drive':
                        # GDrive function returns True/False
                        upload_successful = perform_google_drive_upload(file_path, file_name, target_folder_id_str)
                        # result_info remains None for GDrive unless you want to store something else

                    # Update counts
                    if upload_successful: upload_success_count += 1
                    else: upload_fail_count += 1

                except Exception as e:
                    print(f"❌ UNEXPECTED ERROR during upload loop for {file_name}: {type(e).__name__}")
                    upload_fail_count += 1
                    progress_bar.bar_style = 'danger'

                # time.sleep(0.5) # Optional pause

            # Loop Finished - Final Report
            print("\n" + "=" * 30 + "\n      Upload Summary\n" + "=" * 30)
            print(f"Total Files Attempted: {total_files}\nSuccessful Uploads:    {upload_success_count}\nFailed Uploads:        {upload_fail_count}\n" + "=" * 30)

        finally:
             print("\n--- All uploads finished or failed (Cleanup). ---")
             time.sleep(1)
             progress_bar.layout.display = 'none'; progress_bar.value = 0; progress_bar.bar_style = 'info'
             confirm_and_upload_button.disabled = False
             google_auth_button.disabled = (upload_destination_dropdown.value != 'Google Drive')


# ======================================================
#  Google Drive Auth Button Handler
# ======================================================
# (Unchanged)
def handle_google_auth_click(b):
    global google_auth_button, upload_destination_dropdown
    google_auth_button.disabled = True; authenticate_google_drive()
    google_auth_button.disabled = (upload_destination_dropdown.value != 'Google Drive')

# ======================================================
#  Widget Creation
# ======================================================
# (Unchanged)
output_area = widgets.Output()
current_path_label = widgets.HTML(value=f"<b>Current:</b> {html.escape(current_path)}")
dir_list_box = widgets.VBox([], layout=widgets.Layout(width='100%', max_height='350px', overflow_y='auto', border='1px solid #ccc', padding='5px')) # Adjusted height
select_all_button = widgets.Button(description="Select All", button_style='info', layout=widgets.Layout(width='auto', margin='0 5px 0 0'), tooltip="Select all files currently listed", disabled=True)
deselect_all_button = widgets.Button(description="Deselect All", button_style='info', layout=widgets.Layout(width='auto'), tooltip="Deselect all files currently listed", disabled=True)
select_all_button.on_click(handle_select_all); deselect_all_button.on_click(handle_deselect_all)
select_buttons_box = widgets.HBox([select_all_button, deselect_all_button])
selected_files_viewer = widgets.SelectMultiple(options=[],value=[],description='Selected:',rows=5,disabled=False,layout=widgets.Layout(width='95%', min_height='80px'))
progress_bar = widgets.FloatProgress(value=0, min=0, max=100.0, description='Progress:', bar_style='info', orientation='horizontal', layout=widgets.Layout(width='95%', height='25px', display='none'))
upload_destination_dropdown = widgets.Dropdown(options=['Upload.ee', 'Google Drive'], value='Upload.ee', description='Destination:', disabled=False, layout=widgets.Layout(width='auto'))
gdrive_folder_id_input = widgets.Text(value='', placeholder='Optional: GDrive Folder ID', description='GDrive Folder:', disabled=True, layout=widgets.Layout(width='auto', flex='1'))
google_auth_button = widgets.Button(description="Authenticate GDrive", button_style='info', icon='google', disabled=True, layout=widgets.Layout(width='auto', margin='0 0 0 10px'), tooltip='Authorize access to Google Drive')
google_auth_button.on_click(handle_google_auth_click)
confirm_and_upload_button = widgets.Button(description="Upload Selected Files", button_style='success', icon='upload', layout=widgets.Layout(width='auto'), tooltip='Upload all selected files')
confirm_and_upload_button.on_click(handle_confirm_and_upload_click)

def on_destination_change(change): # Handles enabling/disabling GDrive widgets
    is_gdrive = (change['new'] == 'Google Drive'); gdrive_folder_id_input.disabled = not is_gdrive; google_auth_button.disabled = not is_gdrive
    if is_gdrive and not drive_authenticated: google_auth_button.disabled = False
    elif not is_gdrive: google_auth_button.disabled = True
upload_destination_dropdown.observe(on_destination_change, names='value')

# ======================================================
#  Layout
# ======================================================
# (Unchanged)
path_hbox = widgets.HBox([current_path_label], layout=widgets.Layout(align_items='center', margin='0 0 5px 0'))
destination_box = widgets.HBox([upload_destination_dropdown, gdrive_folder_id_input, google_auth_button], layout=widgets.Layout(align_items='center', flex_flow='row wrap', margin='5px 0'))
file_list_area = widgets.VBox([dir_list_box, select_buttons_box])
upload_control_area = widgets.VBox([selected_files_viewer, destination_box, progress_bar, confirm_and_upload_button], layout=widgets.Layout(margin='10px 0 0 0'))
gui_layout = widgets.VBox([ widgets.HTML("<b>File Browser & Uploader (Multi-Select)</b>"), path_hbox, file_list_area, widgets.HTML("<hr>"), upload_control_area, widgets.HTML("<hr><b>Messages & Upload Log:</b>"), output_area ], layout=widgets.Layout(border='1px solid black', padding='10px', width='95%', max_width='900px'))

# ======================================================
#  Initial Display
# ======================================================
# (Unchanged)
print("Initializing file browser...")
if not google_libs_available:
    print("\n *** Google Drive functionality disabled (libraries not found) *** \n")
    upload_destination_dropdown.options = ['Upload.ee']; upload_destination_dropdown.value = 'Upload.ee'
    gdrive_folder_id_input.disabled = True; google_auth_button.disabled = True
update_directory_view(current_path); update_selected_files_viewer(); display(gui_layout)
print("File browser ready.")
# --- End of Script ---

Initializing file browser...


File browser ready.


In [ ]:
# Block 1: Install necessary libraries
!pip install --upgrade google-api-python-client google-auth-oauthlib google-auth-httplib2 oauth2client pyngrok


#          Colab Script: Upload Google Drive Video to YouTube
# =====================================================================
#
# Instructions for Setup (Read Carefully!):
#
# 0. Google Drive API (IMPORTANT FOR URL UPLOAD):
#    - The feature to upload using a Google Drive share URL requires the
#      Google Drive API to be enabled in your Google Cloud Project.
#    - **ACTION REQUIRED:**
#      a. Go to your Google Cloud Console: https://console.cloud.google.com/
#      b. Make sure you are in the correct project (Project ID: gen-lang-...).
#      c. Navigate to "APIs & Services" -> "Library".
#      d. Search for "Google Drive API".
#      e. Click on it and ensure it is **ENABLED**. If not, click "ENABLE".
#      f. Wait 5-10 minutes after enabling before running the script if using the URL feature.
#
#     - **ACTION REQUIRED (YouTube API)** :
#       a. Go to your Google Cloud Console: https://console.cloud.google.com/
#       b. Make sure you are in the correct project (Project ID: gen-lang-client-...).
#       c. Navigate to "APIs & Services" -> "Library".
#       d. Search for "YouTube Data API v3".
#       e. Click on it.
#       f. Ensure it is ENABLED. If not, click the "ENABLE" button.

# 1. Ngrok Authtoken:
#    - Needed for reliable authentication in Colab.
#    - **ACTION REQUIRED:**
#      a. Get token: https://dashboard.ngrok.com/get-started/your-authtoken
#      b. Paste into `NGROK_AUTHTOKEN` variable below.
#      c. (Optional/Safer: Use Colab Secrets named NGROK_AUTHTOKEN).
#
# 2. Ngrok Fixed Domain (Recommended):
#    - Makes adding Redirect URI to Google easier.
#    - **ACTION REQUIRED:**
#      a. Set your fixed domain (e.g., "my-tunnel.ngrok-free.app") in `NGROK_DOMAIN`.
#      b. If none, set `NGROK_DOMAIN = ""` (uses random URL - impractical).
#
# 3. Google Cloud Client Secrets:
#    - **ACTION REQUIRED:**
#      a. Use/Create an **OAuth 2.0 Client ID** of type **Web application** in GCP.
#      b. Add `https://<YOUR_NGROK_DOMAIN>` (e.g., `https://domain.ngrok-free.app`)
#         to its "Authorized redirect URIs".
#      c. Download the JSON credentials file.
#      d. Copy the *entire content* of the JSON file.
#      e. Paste it between the triple quotes (`"""..."""`) for `CLIENT_SECRETS_CONTENT` below.
#
# 4. Run the Script:
#    - Choose your input method (Path/URL or Folder Selection) when prompted.
#    - Follow authentication steps (visit URL, grant permissions for YouTube AND Drive).
#
# 5. History File:
#    - Upload history saved to `MyDrive/colab_youtube_upload_history.csv`. Change `HISTORY_CSV_FILENAME` if desired.
# ==============================================================================

In [52]:
import os
import pickle
import json
import mimetypes
import sys
import re
import io
import csv
import pandas as pd
from datetime import datetime
import pytz # For timezone
from google.colab import drive
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from googleapiclient.errors import HttpError
import time
import traceback
from pyngrok import ngrok, conf
import threading
import http.server
import socketserver
from urllib.parse import urlparse, parse_qs
import webbrowser

# --- Configuration ---
# *** ACTION REQUIRED: Set values below based on instructions above ***
NGROK_AUTHTOKEN = "" # <-- PASTE NGROK TOKEN
NGROK_DOMAIN = "domain.ngrok-free.app" # <-- SET NGROK DOMAIN (or "")
CLIENT_SECRETS_CONTENT = """
""" # <-- PASTE CLIENT SECRET JSON CONTENT

# Other configurations
SCOPES = [ 'https://www.googleapis.com/auth/youtube.upload', 'https://www.googleapis.com/auth/drive.readonly' ]
API_SERVICE_NAME_YOUTUBE = 'youtube'
API_VERSION_YOUTUBE = 'v3'
API_SERVICE_NAME_DRIVE = 'drive'
API_VERSION_DRIVE = 'v3'
CREDENTIALS_PICKLE_FILE = 'token.pickle'
LOCAL_SERVER_PORT = 5000
TEMP_DOWNLOAD_DIR = '/tmp/colab_downloads'
VIDEO_EXTENSIONS = ('.mp4', '.mov', '.avi', '.wmv', '.flv', '.mkv', '.webm', '.mpeg', '.mpg')
HISTORY_CSV_FILENAME = "colab_youtube_upload_history.csv"
HISTORY_CSV_PATH = None
TIMEZONE = 'UTC'

# --- End Configuration ---


# --- Global variables ---
auth_code = None
auth_state_received = None
server_shutdown_event = threading.Event()

# --- Temporary HTTP Server Handler ---
class _RedirectHandler(http.server.BaseHTTPRequestHandler):
    expected_state = None
    def do_GET(self):
        global auth_code, auth_state_received, server_shutdown_event
        response_message = b"<html><body>Auth Issue</body></html>"
        try:
            parsed_url = urlparse(self.path)
            query_params = parse_qs(parsed_url.query)
            received_code = query_params.get('code', [None])[0]
            received_state = query_params.get('state', [None])[0]
            print(f"\n[TS] Callback received.")
            if received_code:
                auth_code = received_code
                auth_state_received = received_state
                print("[TS] Code captured.")
                response_message = b"<html><body>Auth OK! Return to Colab.</body></html>"
            else:
                error = query_params.get('error', ['Unknown error'])[0]
                print(f"[TS] Error: {error}")
                response_message = f"<html><body>Auth Fail: {error}</body></html>".encode('utf-8')
        except Exception as e:
             print(f"\n[TS] Handler Error: {e}")
             traceback.print_exc()
        finally:
            # Ensure response is sent before signaling shutdown
            try:
                self.send_response(200)
                self.send_header("Content-type", "text/html")
                self.end_headers()
                self.wfile.write(response_message)
            except Exception as send_e:
                 print(f"[TS] Response send error: {send_e}")
            finally:
                 print("[TS] Signaling shutdown...")
                 server_shutdown_event.set() # Signal shutdown AFTER sending response

    def log_message(self, format, *args):
        pass # Keep logging minimal

# --- Function to run the temporary server ---
def run_temp_server(port):
    global server_shutdown_event
    httpd = None
    try:
        socketserver.TCPServer.allow_reuse_address = True
        httpd = socketserver.TCPServer(("", port), _RedirectHandler) # Assign httpd here
        print(f"[TS] Serving on port {port}...")
        while not server_shutdown_event.is_set():
            httpd.handle_request() # Handle one request then check event
        print("[TS] Shutdown event received.")
    except OSError as e:
        print(f"\n[TS] ERROR starting server: {e}")
        server_shutdown_event.set() # Ensure event is set if server fails
    except Exception as e:
        print(f"\n[TS] Unexpected server error: {e}")
        server_shutdown_event.set() # Ensure event is set on other errors
    finally:
        if httpd: # Check if httpd was successfully created before trying to close
            httpd.server_close() # Explicitly close the server socket
            print("[TS] Server socket closed.")
        print("[TS] Thread finished.")

# --- Ngrok Setup Function ---
def setup_ngrok():
    global NGROK_AUTHTOKEN, LOCAL_SERVER_PORT, NGROK_DOMAIN
    print("\n--- Setting up Ngrok ---")
    public_url = None
    try:
        print("Checking existing tunnels...")
        tunnels = ngrok.get_tunnels()
        if tunnels:
            for tunnel in tunnels: # Corrected loop structure
                print(f"Closing tunnel: {tunnel.public_url}")
                try: # Corrected try/except structure
                    ngrok.disconnect(tunnel.public_url)
                except Exception as e:
                    print(f"  Disconnect error: {e}")
            print("Closed existing tunnels.")
        else:
            print("No existing tunnels.")

        if not NGROK_AUTHTOKEN:
            raise ValueError("NGROK_AUTHTOKEN empty.")
        conf.get_default().auth_token = NGROK_AUTHTOKEN
        print("Ngrok auth configured.")

        connect_args = {"addr": LOCAL_SERVER_PORT, "proto": "http"}
        if NGROK_DOMAIN:
            print(f"Connecting Ngrok (fixed domain): {NGROK_DOMAIN} -> {LOCAL_SERVER_PORT}...")
            connect_args["domain"] = NGROK_DOMAIN
        else:
            print(f"Connecting Ngrok (random domain) -> {LOCAL_SERVER_PORT}...")
            print("WARNING: Random URL needs update in GCP!")

        ngrok_tunnel = ngrok.connect(**connect_args)
        public_url = ngrok_tunnel.public_url
        print(f"Ngrok OK: {public_url} -> http://localhost:{LOCAL_SERVER_PORT}")
        if NGROK_DOMAIN and (NGROK_DOMAIN not in public_url):
            print(f"WARN: URL domain != NGROK_DOMAIN config.")
        return public_url
    except Exception as e: # Corrected except block
        print(f"ERROR Ngrok setup: {e}")
        if "account plan" in str(e).lower() or "custom domain" in str(e).lower():
             print("Fixed domain usage might depend on your Ngrok plan.")
        elif "authentication failed" in str(e):
             print("NGROK_AUTHTOKEN may be invalid.")
        traceback.print_exc()
        return None # Return None on failure

# --- Authentication Function ---
def get_authenticated_services():
    global auth_code, auth_state_received, server_shutdown_event, LOCAL_SERVER_PORT, NGROK_DOMAIN, SCOPES
    creds = None
    # Load creds
    if os.path.exists(CREDENTIALS_PICKLE_FILE):
        try:
            with open(CREDENTIALS_PICKLE_FILE, 'rb') as token:
                creds = pickle.load(token)
            print("Loaded creds from pickle.")
            # Check scopes
            if not all(s in creds.scopes for s in SCOPES):
                print("Pickle missing scopes. Re-auth.")
                creds = None # Force re-auth
        except Exception as e:
            print(f"Pickle load fail: {e}")
            creds = None # Force re-auth on error
    # Validate or Refresh
    if creds and not creds.valid:
        if creds.expired and creds.refresh_token:
            print("Refreshing creds...")
            try:
                creds.refresh(Request())
                print("Refresh OK.")
                # Save refreshed token immediately
                with open(CREDENTIALS_PICKLE_FILE, 'wb') as token:
                     pickle.dump(creds, token)
                print("Saved refreshed creds to pickle.")
            except Exception as e:
                print(f"Refresh failed: {e}")
                creds = None # Force re-auth if refresh fails
        else:
            print("Invalid creds found. Re-auth.")
            creds = None # Force re-auth
    # Authenticate if needed
    if not creds:
        print("\n--- Starting OAuth Flow (YouTube & Drive Permissions) ---")
        public_url = None
        server_thread = None
        try:
            public_url = setup_ngrok()
            if not public_url:
                raise Exception("Ngrok fail.")
            # Ensure redirect URI is HTTPS
            redirect_uri = public_url if public_url.startswith("https://") else "https://" + public_url.split("://")[-1]
            print(f"Using Redirect URI: {redirect_uri}")
            if NGROK_DOMAIN:
                 parsed_redirect = urlparse(redirect_uri)
                 if NGROK_DOMAIN != parsed_redirect.netloc:
                      print(f"WARNING: Redirect domain '{parsed_redirect.netloc}' != config '{NGROK_DOMAIN}'!")

            auth_code = None
            auth_state_received = None
            server_shutdown_event.clear() # Reset event
            server_thread = threading.Thread(target=run_temp_server, args=(LOCAL_SERVER_PORT,), daemon=True)
            server_thread.start()
            time.sleep(2) # Allow server thread to start
            if not server_thread.is_alive() or server_shutdown_event.is_set():
                raise Exception("Temp server fail.")

            client_config = json.loads(CLIENT_SECRETS_CONTENT)
            flow = InstalledAppFlow.from_client_config(client_config, SCOPES) # Use all scopes
            flow.redirect_uri = redirect_uri # Tell flow where to expect redirect

            auth_url, state = flow.authorization_url(access_type='offline', prompt='consent')
            _RedirectHandler.expected_state = state
            print("\n" + "="*60 + "\nACTION REQUIRED (Grant YouTube AND Drive Access):\n" + f"1. Ensure '{redirect_uri}' is Auth Redirect URI in GCP.\n" + f"2. Visit URL:\n   {auth_url}\n" + "3. Log in, grant permissions.\n" + f"4. Google redirects to {redirect_uri}.\n" + f"5. Waiting for callback...\n" + "="*60 + "\n")
            try:
                webbrowser.open(auth_url, new=1, autoraise=True)
            except webbrowser.Error:
                print("(Browser open failed)")

            shutdown_confirmed = server_shutdown_event.wait(timeout=300) # 5 min timeout
            if not shutdown_confirmed:
                print("\nERROR: Timeout waiting for auth callback.")
                return None, None # Return None for both services
            if auth_code is None:
                print("\nERROR: No auth code captured.")
                return None, None
            # Validate State
            if state != auth_state_received:
                print(f"\nERROR: State mismatch! Aborting.")
                return None, None
            print("State OK.")
            print("Fetching token...")
            flow.fetch_token(code=auth_code)
            creds = flow.credentials
            print("Auth OK!")
            # Save the newly obtained credentials
            with open(CREDENTIALS_PICKLE_FILE, 'wb') as token:
                pickle.dump(creds, token)
            print("Saved new creds to pickle.")
        except json.JSONDecodeError: # Corrected except block
            print("ERROR: Bad CLIENT_SECRETS_CONTENT.")
            return None, None
        except Exception as e: # Corrected except block
            print(f"ERROR OAuth flow: {e}")
            traceback.print_exc()
            return None, None
        finally: # Corrected finally block
            # Cleanup Ngrok tunnel first
            if public_url:
                try:
                    print(f"Closing Ngrok: {public_url}")
                    ngrok.disconnect(public_url)
                except Exception as ng_e:
                    print(f"Ngrok close error: {ng_e}")
            # Ensure server thread is joined
            if server_thread and server_thread.is_alive():
                print("Waiting server thread...")
                server_thread.join(timeout=5)
                if server_thread.is_alive(): # Check after join
                    print("Warn: Server thread didn't join.")

    # Build services if creds are now valid
    try:
        if not creds:
            print("Auth failed, cannot build services.")
            return None, None
        # Final scope check
        if not all(s in creds.scopes for s in SCOPES):
             print(f"ERROR: Creds missing scopes ({SCOPES}). Re-auth.")
             return None, None
        youtube = build(API_SERVICE_NAME_YOUTUBE, API_VERSION_YOUTUBE, credentials=creds)
        drive_service = build(API_SERVICE_NAME_DRIVE, API_VERSION_DRIVE, credentials=creds)
        print("YouTube and Drive services OK.")
        return youtube, drive_service
    except Exception as e: # Corrected except block
        print(f"Service build failed: {e}")
        return None, None

# --- Function to Parse Google Drive URL ---
def parse_drive_url(url):
    match = re.search(r'/(?:file/d/|open\?id=|uc\?id=)([\w-]+)', url)
    if match:
        return match.group(1)
    return None

# --- Function to Download Google Drive File ---
def download_drive_file(drive_service, file_id, destination_folder):
    if not drive_service:
        print("Drive service unavailable.")
        return None
    destination_path = None
    fh = None
    try:
        print(f"Getting GDrive meta: {file_id}")
        file_metadata = drive_service.files().get(fileId=file_id, fields='id, name, mimeType, size', supportsAllDrives=True).execute()
        file_name = file_metadata.get('name', f"{file_id}_dl")
        file_size = int(file_metadata.get('size', 0))
        print(f"Found: '{file_name}' ({file_size / (1024*1024):.2f} MB)")
        os.makedirs(destination_folder, exist_ok=True)
        destination_path = os.path.join(destination_folder, file_name)
        print(f"Downloading to: {destination_path}")
        request = drive_service.files().get_media(fileId=file_id, supportsAllDrives=True)
        fh = io.FileIO(destination_path, 'wb')
        downloader = MediaIoBaseDownload(fh, request, chunksize=10*1024*1024)
        done = False
        last_progress = -1
        while not done:
            status, done = downloader.next_chunk()
            if status:
                progress = int(status.progress() * 100)
                if progress > last_progress:
                    print(f"\r  Downloading Drive... {progress}%", end="")
                    last_progress = progress
        print("\r  Download Drive complete.      ")
        fh.close()
        fh = None # Indicate file is closed
        return destination_path
    except HttpError as error: # Corrected except block
        print(f'\nDrive DL error: {error}')
        if error.resp.status == 403 and 'accessNotConfigured' in str(error.content):
             print("\n*** Google Drive API Error ***")
             print("Enable API: Search 'Google Drive API' in GCP Library & ENABLE.")
        return None
    except Exception as e: # Corrected except block
        print(f"\nUnexpected Drive DL error: {e}")
        traceback.print_exc()
        return None
    finally: # Corrected finally block
        if fh and not fh.closed:
            fh.close()
        # Clean up partial download if 'done' is not True
        if destination_path and not ('done' in locals() and done) and os.path.exists(destination_path):
             print(f"Cleaning incomplete download: {destination_path}")
             try: # Corrected try/except block
                 os.remove(destination_path)
             except Exception as clean_e:
                 print(f" Error cleaning: {clean_e}")

# --- Upload Function ---
def upload_video(youtube, file_path, title, description, category_id, tags, privacy_status, made_for_kids=False):
    """Uploads a single video file to YouTube. Returns video ID on success, None on failure."""
    try:
        if not file_path or not os.path.exists(file_path):
            print(f"Error: Upload path invalid: '{file_path}'")
            return None
        if not os.path.isfile(file_path):
            print(f"Error: Upload path not file: '{file_path}'")
            return None
        mime_type, _ = mimetypes.guess_type(file_path)
        if not mime_type or not mime_type.startswith('video/'):
            mime_type = 'application/octet-stream'
            print(f"Warn: MIME unknown: {os.path.basename(file_path)}. Using '{mime_type}'.")
        print(f"Starting YT upload: {os.path.basename(file_path)}")
        print(f"  Title: {title}")
        print(f"  Privacy: {privacy_status}")
        body = {'snippet': {'title': title, 'description': description, 'tags': tags, 'categoryId': category_id}, 'status': {'privacyStatus': privacy_status, 'selfDeclaredMadeForKids': made_for_kids,}}
        media_body = MediaFileUpload(file_path, mimetype=mime_type, chunksize=-1, resumable=True)
        request = youtube.videos().insert(part=",".join(body.keys()), body=body, media_body=media_body)
        response = None
        last_progress = -1
        print("  Uploading...", end="")
        while response is None:
            status = None # Initialize status for this iteration
            try:
                status, response = request.next_chunk()
            except HttpError as e: # Corrected except block
                print(f"\nChunk upload HTTP error: {e}")
                raise e # Re-raise
            except Exception as e: # Corrected except block
                print(f"\nChunk upload unexpected error: {e}")
                raise e # Re-raise
            # Process status only if it's not None
            if status:
                progress = int(status.progress() * 100)
                if progress > last_progress:
                    print(f"\r  Uploading... {progress}%", end="")
                    last_progress = progress
        print("\r  Upload completed.                 ")
        video_id = response.get('id')
        if video_id:
            print(f"  Success! YT ID: {video_id} (https://youtu.be/{video_id})")
            return video_id
        else:
            print("  Error: No YT ID received.")
            print(f"  API Response: {response}")
            return None
    except HttpError as e: # Corrected except block
        error_content = "Unknown"
        try:
            error_content = e.content.decode('utf-8')
        except Exception:
             error_content = str(e.content)
        print(f"\n  Upload HTTP error {e.resp.status}:\n  {error_content}")
        if e.resp.status == 403 and 'quotaExceeded' in error_content:
             print("\n *** QUOTA EXCEEDED ***")
        return None
    except FileNotFoundError: # Corrected except block
        print(f"\n  Error: Upload file '{file_path}' not found.")
        return None
    except Exception as e: # Corrected except block
        print(f"\n  Unexpected upload error: {e}")
        traceback.print_exc()
        return None

# --- File Listing Function ---
def list_video_files(directory_path):
    video_files = []
    print(f"\nScanning dir: '{directory_path}'")
    try:
        if not os.path.isdir(directory_path):
            print(f"Error: Not dir: '{directory_path}'")
            return []
        for filename in os.listdir(directory_path):
            full_path = os.path.join(directory_path, filename)
            if os.path.isfile(full_path) and filename.lower().endswith(VIDEO_EXTENSIONS):
                video_files.append(full_path)
        if not video_files:
            print("No videos found.")
        else:
            print(f"Found {len(video_files)} video(s).")
    except Exception as e: # Corrected except block
        print(f"Error scan dir '{directory_path}': {e}")
        return []
    return video_files

# --- User Selection Function ---
def select_videos_from_list(video_files):
    if not video_files:
        return []
    print("\nPlease select video(s):")
    for i, filepath in enumerate(video_files):
        print(f"  {i+1}: {os.path.basename(filepath)}")
    selected_indices = []
    while True:
        user_input = input(f"Enter number(s) (e.g., 1,3), 'all', or blank to cancel: ").strip()
        if not user_input:
            print("Cancelled.")
            return []
        if user_input.lower() == 'all':
            selected_indices = list(range(len(video_files)))
            break
        try:
            raw_indices = [int(x.strip()) for x in user_input.split(',')]
            selected_indices = []
            valid = True
            for index in raw_indices:
                if 1 <= index <= len(video_files):
                    selected_indices.append(index - 1)
                else:
                    print(f"Error: Invalid num '{index}'.")
                    valid = False
                    break
            if valid:
                selected_indices = sorted(list(set(selected_indices)))
                break
        except ValueError: # Corrected except block
            print("Error: Invalid input.")
    selected_files = [video_files[i] for i in selected_indices]
    print(f"\nSelected {len(selected_files)}:")
    # Changed to simple loop for clarity and correct syntax
    for f in selected_files:
        print(f"  - {os.path.basename(f)}")
    return selected_files

# --- History Functions ---
def append_to_history(original_input, status, youtube_id, title, error_msg):
    global HISTORY_CSV_PATH, TIMEZONE
    if not HISTORY_CSV_PATH:
        print("History path not set, cannot save.")
        return
    try:
        file_exists = os.path.isfile(HISTORY_CSV_PATH)
        timestamp = datetime.now(pytz.timezone(TIMEZONE)).strftime('%Y-%m-%d %H:%M:%S %Z')
        row = [timestamp, original_input or "", status or "", youtube_id or '', title or '', error_msg or '']

        with open(HISTORY_CSV_PATH, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            if not file_exists or os.path.getsize(HISTORY_CSV_PATH) == 0:
                writer.writerow(['Timestamp', 'Original Input', 'Status', 'YouTube ID', 'Title', 'Error'])
            writer.writerow(row)
    except Exception as e: # Corrected except block
        print(f"\nERROR writing history '{HISTORY_CSV_PATH}': {e}")

def display_history():
    global HISTORY_CSV_PATH
    print("\n" + "=" * 50 + "\n--- Upload History ---")
    if not HISTORY_CSV_PATH:
        print("History path not set.")
        return
    try:
        if os.path.exists(HISTORY_CSV_PATH):
            df = pd.read_csv(HISTORY_CSV_PATH)
            pd.set_option('display.max_rows', 500)
            pd.set_option('display.max_columns', 50)
            pd.set_option('display.width', 1000)
            pd.set_option('display.max_colwidth', 100)
            print(df.to_string(index=False))
        else:
            print(f"History file not found ({HISTORY_CSV_PATH}).")
    except pd.errors.EmptyDataError:
         print(f"History file ({HISTORY_CSV_PATH}) is empty.")
    except Exception as e: # Corrected except block
        print(f"ERROR reading history '{HISTORY_CSV_PATH}': {e}")
    print("=" * 50)


# --- Main Execution Logic ---
drive_base_path = None; youtube_service = None; drive_service = None
temp_file_path_map = {}; upload_list = []; upload_metadata = {}; results_log = []

try:
    print("--- Mounting Google Drive ---")
    drive.mount('/content/drive', force_remount=True)
    print("Drive mounted.")
    drive_base_path = '/content/drive/MyDrive'
    if not os.path.isdir(drive_base_path):
        drive_base_path = '/content/drive/My Drive'
    if not os.path.isdir(drive_base_path):
        raise Exception("Could not find Drive path.")
    print(f"Drive base path: {drive_base_path}")
    HISTORY_CSV_PATH = os.path.join(drive_base_path, HISTORY_CSV_FILENAME)
    print(f"History path: {HISTORY_CSV_PATH}")

    print("\n--- Choose Input Method ---")
    print("1: Enter comma-separated file paths or Google Drive URLs")
    print("2: Select file(s) from a folder")
    print("3: Upload history")
    choice = input("Enter choice (1 or 2, 3): ").strip()

    print("\n--- Authenticating (YouTube & Drive) ---")
    youtube_service, drive_service = get_authenticated_services()
    if not youtube_service or not drive_service:
        raise Exception("Authentication failed.")

    source_list = []
    if choice == '1':
        print("\n--- Method 1: Enter Paths/URLs ---")
        user_input_str = input("Enter comma-separated FULL paths or Google Drive share URLs:\n").strip()
        if user_input_str:
            source_list = [item.strip() for item in user_input_str.split(',') if item.strip()]
        else:
             print("No input provided.")
    elif choice == '2':
        print("\n--- Method 2: Select From Folder ---")
        relative_dir_path = input(f"Enter directory RELATIVE to '{drive_base_path}': ").strip()
        target_directory = os.path.join(drive_base_path, relative_dir_path)
        available_videos = list_video_files(target_directory)
        source_list = select_videos_from_list(available_videos)
    elif choice == '3':
        print("\n--- Method 3: Upload History ---")
        #display_history()
    else:
        print("Invalid choice.")

    if source_list:
        print(f"\n--- Processing {len(source_list)} Input(s) ---")
        for i, item_input in enumerate(source_list):
            print(f"\nProcessing item {i+1}/{len(source_list)}: '{item_input}'")
            drive_file_id = parse_drive_url(item_input)
            video_path_to_process = None
            status = "Failed - Input Processing"; error_msg = "Unknown processing error"

            if drive_file_id:
                print(f"  Input is Google Drive URL (ID: {drive_file_id}). Downloading...")
                temp_path = download_drive_file(drive_service, drive_file_id, TEMP_DOWNLOAD_DIR)
                if temp_path and os.path.exists(temp_path):
                    video_path_to_process = temp_path; temp_file_path_map[item_input] = temp_path
                    status = "Ready (Downloaded)"; error_msg = ""; print("  Download OK.")
                else:
                    print("  Download failed."); status = "Failed - Download Error"; error_msg = "Could not download file from Google Drive URL."
            elif os.path.exists(item_input):
                if os.path.isfile(item_input): print("  Input is valid file path."); video_path_to_process = item_input; status = "Ready"; error_msg = ""
                else: print(f"  ERROR: Path exists but is not file."); status = "Failed - Not A File"; error_msg = "Input path points to a directory, not a file."
            else: print(f"  ERROR: Input not valid URL or path."); status = "Failed - Invalid Input"; error_msg = "Input is not a recognized Google Drive URL or existing file path."

            if video_path_to_process:
                upload_list.append(video_path_to_process)
                results_log.append({'original_input': item_input, 'final_path': video_path_to_process, 'status': status, 'youtube_id': None, 'title': None, 'error_msg': error_msg})
            else:
                 results_log.append({'original_input': item_input, 'final_path': None, 'status': status, 'youtube_id': None, 'title': None, 'error_msg': error_msg})
                 append_to_history(item_input, status, None, None, error_msg)

    if upload_list:
        print("\n--- Getting Video Details ---")
        items_to_remove = []
        for video_path in upload_list:
            try:
                 print("\n" + "=" * 50 + f"\nEnter details for: {os.path.basename(video_path)}\n" + "=" * 50)
                 base_name = os.path.basename(video_path); suggested_title = os.path.splitext(base_name)[0].replace('_', ' ').replace('-', ' ')
                 video_title = input(f"  Enter title [{suggested_title}]: ") or suggested_title
                 video_description = input("  Enter description: ")
                 video_tags_str = input("  Enter comma-separated tags: ")
                 video_tags = [tag.strip() for tag in video_tags_str.split(',') if tag.strip()]
                 video_category_id = input("  Enter category ID (e.g., 22=People, 27=Education): ").strip()
                 privacy = "private";
                 while privacy not in ['public', 'private', 'unlisted']:
                     privacy = input("  Enter privacy (public, private, unlisted): (private)").lower().strip()
                     if privacy not in ['public', 'private', 'unlisted']: print("    Invalid.")
                 made_for_kids_input = input("  Made for Kids? (yes/no): ").lower().strip()
                 made_for_kids_bool = True if made_for_kids_input == 'yes' else False
                 upload_metadata[video_path] = { 'title': video_title, 'description': video_description, 'tags': video_tags, 'category_id': video_category_id, 'privacy_status': privacy, 'made_for_kids': made_for_kids_bool }
                 for result in results_log:
                      if result['final_path'] == video_path: result['title'] = video_title; break
            except EOFError: # Corrected except block
                 print("\nInput interrupted. Removing video from queue.")
                 items_to_remove.append(video_path)
                 for result in results_log:
                      if result['final_path'] == video_path:
                           result['status'] = "Cancelled - Input"; result['error_msg'] = "Metadata input interrupted."
                           append_to_history(result['original_input'], result['status'], None, None, result['error_msg'])
                           break
                 continue
            except Exception as meta_e: # Corrected except block
                 print(f"\nError getting metadata for {os.path.basename(video_path)}: {meta_e}")
                 items_to_remove.append(video_path)
                 for result in results_log:
                      if result['final_path'] == video_path:
                           result['status'] = "Failed - Metadata Error"; result['error_msg'] = str(meta_e)
                           append_to_history(result['original_input'], result['status'], None, None, result['error_msg'])
                           break
                 continue

        for item in items_to_remove:
             if item in upload_list: upload_list.remove(item)

    if upload_list:
        print("\n--- Starting Upload Process ---")
        success_count = 0; failed_count = 0; total_selected = len(upload_list) # Use upload_list length
        for video_path in upload_list:
            print(f"\n--- Uploading: {os.path.basename(video_path)} ---")
            metadata = upload_metadata.get(video_path)
            if not metadata: print("  Error: Metadata missing. Skipping."); failed_count += 1; continue

            video_id = None; upload_error_msg = None
            try:
                video_id = upload_video( youtube=youtube_service, file_path=video_path, title=metadata['title'], description=metadata['description'], category_id=metadata['category_id'], tags=metadata['tags'], privacy_status=metadata['privacy_status'], made_for_kids=metadata['made_for_kids'] )
            except Exception as upload_e: # Corrected except block
                print(f"  Unexpected critical error during upload call: {upload_e}")
                traceback.print_exc()
                upload_error_msg = str(upload_e)

            status = "Failed - Upload Error"; yt_id_log = None
            if video_id: success_count += 1; status = "Success"; yt_id_log = video_id
            else: failed_count += 1;
            if not upload_error_msg: upload_error_msg = "Upload function returned None (check logs)."

            for result in results_log:
                if result['final_path'] == video_path:
                    result['status'] = status; result['youtube_id'] = yt_id_log; result['error_msg'] = upload_error_msg or ""
                    append_to_history(result['original_input'], result['status'], result['youtube_id'], result['title'], result['error_msg'])
                    break

        print("\n" + "=" * 50 + "\n--- Upload Summary (Current Run) ---")
        print(f"Total Attempted in this run: {total_selected}, Success: {success_count}, Failed (Upload Stage): {failed_count}")
        print("Check history file for detailed status.")
        print("=" * 50)
    else:
         print("\nNo valid videos processed or selected for upload.")

except Exception as e: # Corrected except block
    print("\n--- Unexpected Main Error ---")
    print(e)
    traceback.print_exc()
finally:
    # Cleanup temp files
    if temp_file_path_map:
        print("\n--- Cleaning up temporary downloaded files ---")
        for original, temp_path in temp_file_path_map.items():
            if temp_path and os.path.exists(temp_path):
                try: # Corrected try/except block
                    print(f"Removing temp file for '{original}': {temp_path}")
                    os.remove(temp_path)
                except Exception as clean_e:
                    print(f"Error cleaning temp file {temp_path}: {clean_e}")

    # Display History
    display_history()

    # Final Ngrok Cleanup
    print("\n--- Final Ngrok Cleanup Attempt ---")
    try:
        tunnels = ngrok.get_tunnels()
        if tunnels:
            for tunnel in tunnels: # Corrected loop structure
                print(f"Closing tunnel: {tunnel.public_url}")
                try: # Corrected try/except structure
                    ngrok.disconnect(tunnel.public_url)
                except Exception as dis_e:
                    print(f"  Error disconnecting: {dis_e}")
        else:
            print("No active Ngrok tunnels found.")
    except Exception as e: # Corrected except block
        print(f"Error during final Ngrok cleanup check: {e}")
    print("\n--- Script Finished ---")

--- Mounting Google Drive ---
Mounted at /content/drive
Drive mounted.
Drive base path: /content/drive/MyDrive
History path: /content/drive/MyDrive/colab_youtube_upload_history.csv

--- Choose Input Method ---
1: Enter comma-separated file paths or Google Drive URLs
2: Select file(s) from a folder
3: Upload history
Enter choice (1 or 2, 3): 3

--- Authenticating (YouTube & Drive) ---
Loaded creds from pickle.
YouTube and Drive services OK.

--- Method 3: Upload History ---

No valid videos processed or selected for upload.

--- Upload History ---
              Timestamp                                                    Original Input  Status  YouTube ID                Title                                       Error
2025-04-07 13:54:49 UTC https://drive.google.com/file/d/18rT4MxajPJXjlscsjbMaVxVkihGavGQh Success uUztRTm7XNs Racing Car Animation Upload function returned None (check logs).
2025-04-07 13:54:50 UTC https://drive.google.com/file/d/18rT4MxajPJXjlscsjbMaVxVkihGavGQh Success

In [12]:
!ngrok config add-authtoken ###You token here

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
